## Données "submissions.csv"

### Importation

In [4]:
# importation du training set
import pickle

with open('dataframe.pkl', 'rb') as file:
    df_train = pickle.load(file)
df_train.shape

(2765530, 20)

In [8]:
import pandas as pd
df_test = pd.read_csv('./content/submissions.csv')
df_test.shape

C:\Users\bourh\AppData\Local\Temp\ipykernel_26720\544240494.py:2: DtypeWarning: Columns (9,15,17,19,31) have mixed types. Specify dtype option on import or set low_memory=False.
  df_test = pd.read_csv('./content/submissions.csv')


(378041, 34)

In [9]:
print("Nombre de valeurs manquantes pour type local :", df_test['Type local'].isna().sum())

Nombre de valeurs manquantes pour type local : 50000


### Classification pour prédire notre variable 'Type local'

In [10]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ShuffleSplit

#### Echantillonage

In [28]:
# on prend nos variables explicatives
variables_explicatives = ["Surface reelle bati", "Surface terrain", "Nombre pieces principales", "Nombre de lots","Code commune"]
X_train_classif = df_train[variables_explicatives]

#on nomme la variale cible
y_train_classif = df_train["Type local"]

In [24]:
y_train_classif.value_counts(normalize=True)

Type local
Maison                                      0.539749
Appartement                                 0.270128
Dépendance                                  0.140131
Local industriel. commercial ou assimilé    0.049992
Name: proportion, dtype: float64

In [34]:
X_test_classif = df_test[df_test['Type local'].isna()][variables_explicatives]
X_test_classif

,Surface reelle bati,Surface terrain,Nombre pieces principales,Nombre de lots,Code commune
4,99.0,765.0,5.0,0,53
13,76.0,1134.0,3.0,0,231
33,95.0,342.0,5.0,0,435
40,230.0,627.0,0.0,0,249
44,0.0,NaN,0.0,1,354
...,...,...,...,...,...
378010,34.0,NaN,2.0,1,111
378015,0.0,NaN,0.0,1,111
378019,0.0,NaN,0.0,1,116
378023,10.0,NaN,1.0,1,113


In [39]:
X_train_classif['Nombre pieces principales'].fillna(X_train_classif['Nombre pieces principales'].mean(), inplace=True)
X_test_classif['Nombre pieces principales'].fillna(X_test_classif['Nombre pieces principales'].mean(), inplace=True)
X_train_classif['Surface reelle bati'].fillna(X_train_classif['Surface reelle bati'].mean(), inplace=True)
X_test_classif['Surface reelle bati'].fillna(X_test_classif['Surface reelle bati'].mean(), inplace=True)
X_train_classif["Surface terrain"].fillna(X_train_classif["Surface terrain"].mean(), inplace=True)
X_test_classif["Surface terrain"].fillna(X_test_classif["Surface terrain"].mean(), inplace=True)
X_train_classif["Nombre de lots"].fillna(X_train_classif["Nombre de lots"].mean(), inplace=True)
X_test_classif["Nombre de lots"].fillna(X_test_classif["Nombre de lots"].mean(), inplace=True)
X_train_classif["Code commune"].fillna(X_train_classif["Code commune"].mean(), inplace=True)
X_test_classif["Code commune"].fillna(X_test_classif["Code commune"].mean(), inplace=True)


C:\Users\bourh\AppData\Local\Temp\ipykernel_26720\4136062022.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train_classif['Nombre pieces principales'].fillna(X_train_classif['Nombre pieces principales'].mean(), inplace=True)
C:\Users\bourh\AppData\Local\Temp\ipykernel_26720\4136062022.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train_classif['Surface reelle bati'].fillna(X_train_classif['Surface reelle bati'].mean(), inplace=True)
C:\Users\bourh\AppData\Local\Temp\ipykernel_26720\4136062022.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats

In [40]:
pourcentage_manquant = (X_test_classif.isna().sum() / len(X_test_classif)) * 100
pourcentage_manquant

Surface reelle bati          0.0
Surface terrain              0.0
Nombre pieces principales    0.0
Nombre de lots               0.0
Code commune                 0.0
dtype: float64

In [41]:
parameters = {'max_depth' : [1,2,3,4,5,6] ,
              'min_samples_leaf' : [10,20,30,40],
              'min_samples_split' : [50,100,200]}

dtc = DecisionTreeClassifier()
dtc_model_1 = GridSearchCV(dtc, parameters, scoring = 'accuracy' ,
                   cv = ShuffleSplit(test_size=0.30, n_splits=1, random_state=0))
dtc_model_1.fit(X_train_classif, y_train_classif )

print("Voici les paramètres du meilleure modèle : " + str(dtc_model_1.best_estimator_))
print("Voici le score "  + str(dtc_model_1.scorer_) + " du meilleure modèle : " + str(dtc_model_1.best_score_))

Voici les paramètres du meilleure modèle : DecisionTreeClassifier(max_depth=6, min_samples_leaf=20, min_samples_split=50)
Voici le score make_scorer(accuracy_score) du meilleure modèle : 0.9731817529852627


#### Prédiction

In [47]:
y_pred_classif = dtc_model_1.predict(X_test_classif)
y_pred_classif

array(['Maison', 'Maison', 'Maison', ..., 'Dépendance', 'Appartement',
       'Local industriel. commercial ou assimilé'], dtype=object)

#### Rajout de nos prédictions de 'type local' dans notre df_test 

In [60]:
df_test.loc[df_test['Type local'].isna(), 'Type local'] = y_pred_classif

In [61]:
# Plus de valeur manquantes pour nos 'type local'
df_test['Type local'].isna().sum()

0

### Régression pour prédire notre 'Valeur foncière'

#### Echantillonage

D'après mes tests sur différents modèles, je priviligierais de prendre certaines variables : 'Type local', 'Nombre pieces principales', 'Surface reelle bati', 'Surface terrain' et 'Nombre de lots'

In [81]:
variables_explicatives = ['Type local', 'Nombre pieces principales', 'Surface reelle bati', 'Surface terrain', 'Nombre de lots']

X_train_reg = df_train[variables_explicatives]
X_test_reg = df_test[variables_explicatives]
y_train_reg = df_train['Valeur fonciere']

# conversion des variables catégorielles en variables indicatrices 
X_train_reg = pd.get_dummies(data=X_train_reg, columns=['Type local'])
X_test_reg = pd.get_dummies(data=X_test_reg, columns=['Type local'])

In [82]:
# on remplace les NA dans la colonne 'Nombre pieces principales' et dans 'Surface reelle bati'
# et dans 'Surface terrain' par leurs moyennes
X_train_reg['Nombre pieces principales'].fillna(X_train_reg['Nombre pieces principales'].mean(), inplace=True)
X_test_reg['Nombre pieces principales'].fillna(X_test_reg['Nombre pieces principales'].mean(), inplace=True)
X_train_reg['Surface reelle bati'].fillna(X_train_reg['Surface reelle bati'].mean(), inplace=True)
X_test_reg['Surface reelle bati'].fillna(X_test_reg['Surface reelle bati'].mean(), inplace=True)
X_train_reg['Surface terrain'].fillna(X_train_reg['Surface terrain'].mean(), inplace=True)
X_test_reg['Surface terrain'].fillna(X_test_reg['Surface terrain'].mean(), inplace=True)
X_train_reg['Nombre de lots'].fillna(X_train_reg['Nombre de lots'].mean(), inplace=True)
X_test_reg['Nombre de lots'].fillna(X_test_reg['Nombre de lots'].mean(), inplace=True)

C:\Users\bourh\AppData\Local\Temp\ipykernel_26720\726301858.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train_reg['Nombre pieces principales'].fillna(X_train_reg['Nombre pieces principales'].mean(), inplace=True)
C:\Users\bourh\AppData\Local\Temp\ipykernel_26720\726301858.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test_reg['Nombre pieces principales'].fillna(X_test_reg['Nombre pieces principales'].mean(), inplace=True)
C:\Users\bourh\AppData\Local\Temp\ipykernel_26720\726301858.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the d

In [83]:
pourcentage_manquant = (X_test_reg.isna().sum() / len(X_test_reg)) * 100
pourcentage_manquant

Type local                   0.0
Nombre pieces principales    0.0
Surface reelle bati          0.0
Surface terrain              0.0
Nombre de lots               0.0
dtype: float64

#### Apprentissage - Régression Ridge

D'après mes tests sur différents modèles, je priviligierais d'utiliser le modèle 'Ridge' avec pour paramètre '{'alpha': 1}'

In [85]:
from sklearn.preprocessing import StandardScaler

# Appliquez la standardisation aux données de X
scaler = StandardScaler()
X_train_CR = scaler.fit_transform(X_train_reg)
X_test_CR = scaler.fit_transform(X_test_reg)
pd.DataFrame(X_test_CR, columns=X_test_reg.columns).head(3) 

,Nombre pieces principales,Surface reelle bati,Surface terrain,Nombre de lots,Type local_Appartement,Type local_Dépendance,Type local_Local industriel. commercial ou assimilé,Type local_Maison
0,-0.902082,-0.157903,-1.745973e-17,0.509581,2.062115,-0.405538,-0.297682,-1.192121
1,0.080642,0.081810,-1.745973e-17,0.509581,2.062115,-0.405538,-0.297682,-1.192121
2,1.063366,0.015682,6.704988e-03,-0.466996,-0.484939,-0.405538,-0.297682,0.838841


In [86]:
# Apprentissage

import numpy as np
from sklearn.linear_model import Ridge

parameters = {'alpha': np.arange(start = 0, stop = 10, step = 1)}

ridge_model = Ridge()
grid_ridge = GridSearchCV(ridge_model, parameters, scoring = 'neg_mean_squared_error', verbose = 1)
grid_ridge.fit(X_train_CR, y_train_reg)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


GridSearchCV(estimator=Ridge(),
             param_grid={'alpha': array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])},
             scoring='neg_mean_squared_error', verbose=1)

In [87]:
# Prédiction 

print(pd.DataFrame(grid_ridge.cv_results_).loc[:,['params','mean_test_score']])
print("Meilleur paramètre :", grid_ridge.best_params_)
print("Meilleur score :", grid_ridge.best_score_)
y_pred = grid_ridge.best_estimator_.predict(X_test_CR)

         params  mean_test_score
0  {'alpha': 0}    -1.021840e+10
1  {'alpha': 1}    -1.021840e+10
2  {'alpha': 2}    -1.021840e+10
3  {'alpha': 3}    -1.021840e+10
4  {'alpha': 4}    -1.021840e+10
5  {'alpha': 5}    -1.021840e+10
6  {'alpha': 6}    -1.021840e+10
7  {'alpha': 7}    -1.021840e+10
8  {'alpha': 8}    -1.021840e+10
9  {'alpha': 9}    -1.021840e+10
Meilleur paramètre : {'alpha': 9}
Meilleur score : -10218397222.39857


In [111]:
# dataframe de nos prédictions
df_predictions_val_foncieres = pd.DataFrame(y_pred, index=np.arange(1, len(y_pred)+1), columns=['TARGET']).rename_axis("ID")
df_predictions_val_foncieres

,TARGET
ID,
1,121057.387361
2,165985.563934
3,201966.809291
4,179655.808538
5,201936.846224
...,...
378037,134761.026581
378038,114852.233406
378039,148664.846101


In [113]:
# passe nos prédictions en csv pour les envoyer à kaggle
df_predictions_val_foncieres.to_csv('predictions_valeurs_foncieres.csv')

In [115]:
!kaggle competitions submit m2-sise-2023 -f predictions_valeurs_foncieres.csv -m ""

Successfully submitted to M2-SISE-2023  Sarde Immo



  0%|          | 0.00/9.49M [00:00<?, ?B/s]
  0%|          | 16.0k/9.49M [00:01<13:00, 12.7kB/s]
  2%|▏         | 240k/9.49M [00:01<00:40, 239kB/s]  
 10%|█         | 992k/9.49M [00:01<00:07, 1.16MB/s]
 21%|██        | 1.97M/9.49M [00:01<00:03, 2.40MB/s]
 26%|██▌       | 2.48M/9.49M [00:01<00:02, 2.89MB/s]
 34%|███▍      | 3.27M/9.49M [00:01<00:01, 3.88MB/s]
 41%|████      | 3.86M/9.49M [00:02<00:01, 3.70MB/s]
 57%|█████▋    | 5.39M/9.49M [00:02<00:00, 6.04MB/s]
 71%|███████▏  | 6.78M/9.49M [00:02<00:00, 7.86MB/s]
 82%|████████▏ | 7.73M/9.49M [00:02<00:00, 7.99MB/s]
100%|██████████| 9.49M/9.49M [00:03<00:00, 3.25MB/s]
